### TenDimensions of communications

Pragmatic meaning of communications.

* https://github.com/lajello/tendimensions
* Adaptar los modelos de embeddings para extraer los features.
* Para cada dimensión hay UN modelo con un embedding determinado.
* Los nombres de los embeddings están hardcodeados !
    * Ver el formato para cuando lo quiera adaptar con los de español.
    * Hay que convertir los embeddings al formato de ``Word2VecKeyedVectors``
    * Esto arruina el uso de FastText !! 
* ``compute_score`` calcula score por el texto completo, ``compute_score_split`` calcula por oración y luego promedia
    

##### Modelos para español

* FastText
    * ``wiki-news-300d-1M-subword`` están entrenados con: 1 million word vectors trained with subword infomation on Wikipedia 2017, UMBC webbase corpus and statmt.org news dataset (16B tokens)
    * Son los modelos originales de Facebook, entrenados con Wikipedia o CommonCrawl.
        * Nos quedamos con los de Wikipedia (wiki.es)
    * https://github.com/facebookresearch/fastText/blob/master/docs/pretrained-vectors.md
    * Pre-procesado: lower case
* GloVe
    * ``glove.42B.300d`` están entrenados con CommonCrawl
    * Stanford no provee modelos en español.
    * Usamos los de los chilenos, entrenados con "Spanish Billion Word Corpus". Incluye Wikipedia, pero no CommonCrawl, aunque tiene algunos más.
    * Tienen un problema con las vocales acentuadas, están TODAS en mayúsculas !!
* Word2Vec
    * ``GoogleNews-vectors-negative300`` entrenados con Google News, 3 millones de palabras.
    * Google no provee modelos en español.
    * Usamos los de los chilenos, entrenados con "Spanish Billion Word Corpus". Incluye Wikipedia, pero no CommonCrawl, aunque tiene algunos más.
    * Pre-procesado: hay upper case, y los most_similar no son los mismos
    
Los chilenos de la uchile que entrenaron todos: https://github.com/dccuchile/spanish-word-embeddings
* No tuvieron mucho cuidado al entrenarlos.
* GloVe tiene problemas con los acentos. No todo está en lower case.
    * No es simple de arreglar, los nombres própios están en upper case.
    * Si el encodding es palabra a palabra, lo que se podría hacer es que si no existe, se busque primero la versión lower y si ninguna está que se asigne el UNK. (setear un parámetro ?) --> Ya lo hacía por defecto el código!
* Hacer una combinación? Calcular el valor en case como viene y en lower case? Cómo para tener para comparar.

In [ ]:
import tendims

base_dir = './tendimensions-main/'

embeddings_dir = base_dir + 'embeddings/'
models_dir = base_dir + 'models/lstm_trained_models/' 

In [ ]:
model = tendims.TenDimensionsClassifier(is_cuda=False, models_dir = models_dir, embeddings_dir = embeddings_dir)
model.dimensions_list

In [ ]:
text = 'Hello, my name is Mike. I am willing to help you, whatever you need.'

In [ ]:
print(model.compute_score(text, None))
model.compute_score_split(text, None) #compute a score for each sentence and returns maximum and average values
# (avg, max, min, std)

In [ ]:
sentences = {
'knowledge' : [
    "Only a fully trained Jedi Knight, with The Force as his ally, will conquer Vader and his Emperor. If you end your training now, if you choose the quick and easy path, as Vader did, you will become an agent of evil",
    "Well, in layman's terms, you use a rotating magnetic field to focus a narrow beam of gravitons; these in turn fold space-time consistent with Weyl tensor dynamics until the space-time curvature becomes infinitely large and you have a singularity",
    "Since positronic signatures have only been known to emanate from androids such as myself, it is logical to theorize that there is an android such as myself on Kolarus III",
],

'power' : [
    "Now if you don't want to be the fifth person ever to die in meta-shock from a planar rift, I suggest you get down behind that desk and don't move until we give you the signal",
    "You can ask any price you want, but you must give me those letters ",
    "Right now you're in no position to ask questions! And your snide remarks..."
],

'status' : [
    "I want to thank you, sir, for giving me the opportunity to work",
    "Frankie, you're a good old man, and you've been loyal to my Father for years...so I hope you can explain what you mean",
    "And we drink to her, and we all congratulate her on her wonderful accomplishment during this last year...her great success in A Doll's House!"
],

'trust' : [
    "I'm trying to tell you – and this is where you have to trust me – but, I think your life might be in real danger",
    "Mr. Lebowski is prepared to make a generous offer to you to act as courier once we get instructions for the money",
    "Take the Holy Gospels in your hand and swear to tell the whole truth concerning everything you will be asked"
],

'support' : [
    "I'm sorry, I just feel like... I know I shouldn't ask, I just need some kind of help, I just, I have a deadline tomorrow",
    "Look, Dave, I know that you're sincere and that you're trying to do a competent job, and that you're trying to be helpful, but I can assure the problem is with the AO-units, and with your test gear",
    "Well... listen, if you need any help, you know, back up, call me, OK?"
],

'romance' : [
    "I'm going to marry the woman I love",
    "If you are truly wild at heart, you'll fight for your dreams... Don’t turn away from love, Sailor ",
    "You admit to me you do not love your fiance?"
],

'identity' : [
    "Hey, I know what I'm talkin' about, black women ain't the same as white women ",
    "That's how it was in the old world, Pop, but this is not Sicily",
    "But, as you are so fond of observing, Doctor, I'm not human"
],

'fun' : [
    "It’s just funny...who needs a serial psycho in the woods with a chainsaw when we have ourselves",
    "I do enjoy playing bingo, if you'd like to join me for a game tomorrow night at church you’re welcome to",
    "Oh, I'm sure it’s a lot of fun, 'cause the Incas did it, you know, and-and they-they-they were a million laughs"
],

'conflict' : [
    "Forgive me for askin', son, and I don’t mean to belabor the obvious, but why is it that you’ve got your head so far up your own ass?",
    "If you're lying to me you poor excuse for a human being, I'm gonna blow your brains all over this car",
    "I couldn't give a shit if you believe me or not, and frankly I'm too tired to prove it to you"
]}

In [ ]:
import pandas as pd
rr = []
for dim in sentences:
    for s in sentences[dim]:
        score = model.compute_score_split(s, None)
        score['text'] = s
        score['all'] =  model.compute_score(s, None)
        rr.append(score)
        
pd.DataFrame(rr).to_pickle(base_dir + 'df_results_en_models_.pickle')

In [ ]:
sentences = {
'knowledge' : [
    "Only a fully trained Jedi Knight, with The Force as his ally, will conquer Vader and his Emperor. If you finish your training now, if you choose the quick and easy path, as Vader did, you will become an agent of evil.",
    "Well, in simple terms, you use a rotating magnetic field to focus a narrow beam of gravitons; these, in turn, fold spacetime according to the dynamics of the Weyl tensor until the curvature of spacetime becomes infinitely large and has a singularity",
    "Since positronic signatures are only known to emanate from androids like me, it stands to reason that there is an android like me on Kolarus III.",
],

'power' : [
    "Now, if you don't want to be the fifth person to die in a meta-shock from a flat crack, I suggest you get behind that desk and don't move until we give you the signal.",
    "You can ask the price you want, but you must give me those cards",
    "Right now you are in no condition to ask questions! And your sarcastic comments"
],

'status' : [
    "I want to thank you sir for giving me the opportunity to work",
    "Frankie, you are a good old man and have been loyal to my father for years... so I hope you can explain what you mean.",
    "And we drink to her, and we all congratulate her on her wonderful achievement this past year... her great success in A Doll's House!"
],

'trust' : [
    "I'm trying to tell you, and this is where you have to trust me, but I think your life might be in real danger.",
    "Mr. Lebowski is prepared to make you a generous offer to act as courier once we receive instructions for the money.",
    "Take in your hand the Holy Gospels and swear to tell the whole truth in everything they ask you."
],

'support' : [
    "I'm sorry, I'm sorry that... I know I shouldn't ask, I just need some kind of help, I just have a deadline tomorrow.",
    "Look, Dave, I know you're honest and trying to do a competent job and trying to help, but I can assure you that the problem is with the units and your test.",
    "Well... listen, if you need any help, you know, call me, okay?"
],

'romance' : [
    "I'm going to marry the woman I love.",
    "If you are truly wild at heart, you will fight for your dreams... Don't turn your back on love, Sailor.",
    "Will you admit to me that you don't love your fiancé?"
],

'identity' : [
    "Hey, I know what I'm talking about, black women are not the same as white women.",
    "That's how it was in the old world, Dad, but this isn't Sicily.",
    "But as you like to point out, doctor, I'm not human."
],

'fun' : [
    "It's funny... who needs a serial psycho in the woods with a chainsaw when we've got ourselves?",
    "I enjoy playing bingo, if you'd like to join me for a game tomorrow night at church you're welcome.",
    "Oh, I'm sure it's a lot of fun, because the Incas did it, you know, and they-they-they were a million laughs."
],

'conflict' : [
    "Forgive me for asking, son, and I don't want to stress the obvious, but why do you have your head so far up your own ass?",
    "If you're lying to me, you poor excuse for a human being, I'll blow your brains out all over this car.",
    "I don't give a shit if you believe me or not, and frankly, I'm too tired to show it to you."
]}

In [ ]:
import pandas as pd
rr = []
for dim in sentences:
    for s in sentences[dim]:
        score = model.compute_score_split(s, None)
        score['text'] = s
        score['all'] =  model.compute_score(s, None)
        rr.append(score)
        
pd.DataFrame(rr).to_pickle(base_dir + 'df_results_en_models_translated.pickle')

#### Formato Gensim para los modelos

Gensim requiere que todo esté en el formato de Word2Vec, lo que significa que pierde un poco de capacidad de análisis cuando los embeddings no se calculan por palabra completa, como es el caso de FastText.

In [ ]:
from gensim.models import KeyedVectors # este es para cargar los vectors ...

embeddings_es_dir = './embeddings/'

In [ ]:
# FastText
file_path = embeddings_es_dir + 'wiki.es/wiki.es.vec' 
file_path = embeddings_es_dir + 'wiki.es/wiki.es.align.vec' 

model = KeyedVectors.load_word2vec_format(file_path,binary=False) # estamos levantando el formato texto

model.most_similar('perro') # probamos que funcione

In [ ]:
output_file = embeddings_es_dir + 'wiki.es/wiki.es.align.wv' 
# model.save_word2vec_format(output_file,binary=True) # este no lo deja en el formato que espera el otro !
model.save(output_file)

model = KeyedVectors.load(output_file,mmap='r')

model.most_similar('perro') # son todas lower case !

In [ ]:
# Word2Vec es lo mismo que para FastText

file_path = embeddings_es_dir + 'sbw_vectors.bin' 

model = KeyedVectors.load_word2vec_format(file_path,binary=True) 

model.most_similar('perro') # probamos que funcione

In [ ]:
output_file = embeddings_es_dir + 'sbw_vectors.wv' 
model.save(output_file)

model = KeyedVectors.load(output_file,mmap='r')
model.most_similar('perro') 

In [ ]:
file_path = embeddings_es_dir + 'multi.wiki.bpe.vs1000000.d300.w2v.bin'

model = KeyedVectors.load_word2vec_format(file_path,binary=True) 

model.most_similar('perro') # probamos que funcione

In [ ]:
output_file = embeddings_es_dir + 'multi.wiki.bpe.vs1000000.d300.w2v.wv' 
model.save(output_file)

model = KeyedVectors.load(output_file,mmap='r')
model.most_similar('perro') 

In [ ]:
# Glove 
from gensim.scripts.glove2word2vec import glove2word2vec

file_path = embeddings_es_dir + 'glove-sbwc.i25.vec' 
output_file = embeddings_es_dir + 'glove-sbwc.i25.wv'

model = KeyedVectors.load_word2vec_format(file_path,binary=False) 

model.save(output_file)

model = KeyedVectors.load(output_file,mmap='r')
model.most_similar('perro') 

In [ ]:
# Glove -- NO ES REALMENTE MULTILIGUAL?
from gensim.scripts.glove2word2vec import glove2word2vec

file_path = embeddings_es_dir + 'multilingual_embeddings.es' 
output_file = embeddings_es_dir + 'glove_multilingual_embeddings.es.wv'

glove2word2vec(file_path, output_file) # este es para cuando el archivo no tiene la primera línea de cantidad

model = KeyedVectors.load_word2vec_format(output_file,binary=False) 

model.save(output_file)

model = KeyedVectors.load(output_file,mmap='r')
model.most_similar('perro') 

In [ ]:
# model.most_similar('dog') 

for x in model.key_to_index.keys():
    print(x)
#     if x.lower() != x:
#         print(x,'--',x.lower() in model.key_to_index)

#### Prueba de traducción

La intuición es que la semántica de las relaciones no debería cambiar notablemente si están expresadas en español o en inglés, con lo que si probamos las oraciones con los embeddings en inglés y las traducimos para usar los embeddings en español, no debería haber mucha diferencia.

Otra opción: Usar modelos de embeddings multilingües.

* Hay diferencia para algunas de las dimensiones.
    * Trust, por ejemplo, queda siempre arriba, tanto como si correspondiera (de acuerdo a los resultados en inglés), como si no.
    * Fun, conflict también sufren algunas diferencias.
* Haciendo inglés -> español -> inglés se conservan los scores originales.
    * Traducir los tweets sería mucho?
    
    
* Probar qué pasa con oraciones propias en español y traducidas al inglés.
    * A ver si hay algún tipo de relación entre los scores.
    * A ver si los scores parece que dan como nos interesaría que den.


* Modelos crosslingual:
    * word2vec: 
    * gloVe: https://bpemb.h-its.org/multi/ -- No parece ser realmente multi
    * FastText: en la misma página que los otros.
    * No mejora los resultados :(
   

In [ ]:
import tendims

base_dir = './'

embeddings_dir = base_dir + 'embeddings_multi/'
models_dir = base_dir + 'models/lstm_trained_models/' 

In [ ]:
model = tendims.TenDimensionsClassifier(is_cuda=False, models_dir = models_dir, embeddings_dir = embeddings_dir)
model.dimensions_list

In [ ]:
text = 'Hola, mi nombre es Mike. Estoy dispuesto a ayudarte en lo que necesites.'
print(model.compute_score(text, None))
model.compute_score_split(text, None)

In [ ]:
sentences = {
'knowledge' : [
    "Solo un Caballero Jedi completamente entrenado, con La Fuerza como su aliado, conquistará a Vader y su Emperador. Si terminas tu entrenamiento ahora, si eliges el camino rápido y fácil, como hizo Vader, te convertirás en un agente del mal.",
    "Bueno, en términos sencillos, usas un campo magnético giratorio para enfocar un estrecho haz de gravitones; estos, a su vez, pliegan el espacio-tiempo de acuerdo con la dinámica del tensor de Weyl hasta que la curvatura del espacio-tiempo se vuelve infinitamente grande y tiene una singularidad",
    "Dado que solo se sabe que las firmas positrónicas emanan de androides como yo, es lógico teorizar que hay un androide como yo en Kolarus III.",
],

'power' : [
    "Ahora, si no quieres ser la quinta persona en morir en un meta-shock por una grieta plana, te sugiero que te pongas detrás de ese escritorio y no te muevas hasta que te demos la señal.",
    "Puedes pedir el precio que quieras, pero debes darme esas cartas",
    "¡Ahora mismo no estás en condiciones de hacer preguntas! Y tus comentarios sarcásticos"
],

'status' : [
    "Quiero agradecerle señor por darme la oportunidad de trabajar",
    "Frankie, eres un buen anciano y has sido leal a mi padre durante años... así que espero que puedas explicar lo que quieres decir.",
    "Y brindamos por ella, y todos la felicitamos por su maravilloso logro durante este último año... ¡su gran éxito en Casa de muñecas!"
],

'trust' : [
    "Estoy tratando de decírtelo, y aquí es donde tienes que confiar en mí, pero creo que tu vida podría estar en peligro real.",
    "El Sr. Lebowski está preparado para hacerle una generosa oferta para que actúe como mensajero una vez que recibamos las instrucciones para el dinero.",
    "Toma en tu mano los Santos Evangelios y jura decir toda la verdad en todo lo que te pregunten."
],

'support' : [
    "Lo siento, siento que... Sé que no debería preguntar, solo necesito algún tipo de ayuda, solo tengo una fecha límite mañana.",
    "Mira, Dave, sé que eres sincero y que estás tratando de hacer un trabajo competente, y que estás tratando de ayudar, pero te puedo asegurar que el problema está en las unidades y en tu prueba.",
    "Bueno... escucha, si necesitas ayuda, ya sabes, llámame, ¿de acuerdo?"
],

'romance' : [
    "Me voy a casar con la mujer que amo.",
    "Si eres verdaderamente salvaje de corazón, lucharás por tus sueños... No le des la espalda al amor, Marinero.",
    "¿Me admites que no amas a tu prometido?"
],

'identity' : [
    "Oye, sé de lo que hablo, las mujeres negras no son lo mismo que las mujeres blancas.",
    "Así era en el viejo mundo, papá, pero esto no es Sicilia.",
    "Pero, como le gusta observar, doctor, no soy humano."
],

'fun' : [
    "Es divertido... ¿quién necesita un psicópata en serie en el bosque con una motosierra cuando nos tenemos a nosotros mismos?",
    "Disfruto jugando al bingo, si quieres unirte a mí para un juego mañana por la noche en la iglesia, eres bienvenido.",
    "Oh, estoy seguro de que es muy divertido, porque los Incas lo hicieron, ya sabes, y ellos-ellos-ellos fueron un millón de risas."
],

'conflict' : [
    "Perdóname por preguntar, hijo, y no quiero insistir en lo obvio, pero ¿por qué tienes la cabeza tan metida en tu propio trasero?",
    "Si me estás mintiendo, pobre excusa de ser humano, te volaré los sesos por todo este auto.",
    "Me importa una mierda si me crees o no y, francamente, estoy demasiado cansado para demostrártelo."
]}

In [ ]:
import pandas as pd
rr = []
for dim in sentences:
    for s in sentences[dim]:
        score = model.compute_score_split(s, None)
        score['text'] = s
        score['all'] =  model.compute_score(s, None)
        rr.append(score)
        
pd.DataFrame(rr).to_pickle(base_dir + 'df_results_multi.pickle')

In [ ]:
import pandas as pd

df_en = pd.read_pickle(base_dir + 'df_results_en_models_.pickle')
df_es = pd.read_pickle(base_dir + 'df_results_multi.pickle')

In [ ]:
df_merge = df_en.merge(df_es,left_index=True,right_index=True,suffixes=('_en', '_es'))
df_merge

In [ ]:
from collections import defaultdict

diff = defaultdict(list)

dd = ['support', 'knowledge', 'conflict', 'power', 'similarity', 'fun', 'status', 'trust', 'identity', 'romance']
for i in range(0,len(df_merge)):
    print(df_merge['text_en'].values[i])
#     print(df_merge['all_en'].values[i])
    print(dict(sorted(df_merge['all_en'].values[i].items(), key=lambda item: -item[1])).keys())
#     print(df_merge['all_es'].values[i])
    print(dict(sorted(df_merge['all_es'].values[i].items(), key=lambda item: -item[1])).keys())
    for d in dd:
#         print('  ',d,"::",df_merge[d+'_en'].values[i])
#         print('  ',d,"::",df_merge[d+'_es'].values[i])
        print('  ',d,"::",round(df_merge[d+'_en'].values[i][0],3),' - ',round(df_merge[d+'_es'].values[i][0],3),' == ',round(df_merge[d+'_en'].values[i][0] - df_merge[d+'_es'].values[i][0],3), 
#                           df_merge[d+'_en'].values[i][1] - df_merge[d+'_es'].values[i][1], 
#                           df_merge[d+'_en'].values[i][2] - df_merge[d+'_es'].values[i][2], 
#                           df_merge[d+'_en'].values[i][3] - df_merge[d+'_es'].values[i][3]
             )
        diff[d].append(abs(round(df_merge[d+'_en'].values[i][0] - df_merge[d+'_es'].values[i][0],3)))

In [ ]:
for k,v in diff.items():
    print(k,' :: ',max(v),min(v),' == ',sorted(v))

#### Prueba con algunas traducciones

Ya tenemos algunos tweets traducidos, vamos a probar sobre esos tweets obtener los scores, a ver qué parece.
Necesita ``df_tweets_db`` para poder ver el texto original.

In [ ]:
from collections import deque
from datetime import datetime
from tqdm.notebook import tqdm
import pickle
import pandas as pd
import os

In [ ]:
translated = set()
for file in tqdm(os.listdir('./')):
    if file.startswith('_social_translations_') and not file.startswith('_social_translations_simplified_'):
        translated.update(set(pd.read_pickle(file)['tweet_id']))

print(len(translated))

rr = deque()
for file in tqdm(os.listdir('./')):
    if file.startswith('_translation'):
        with open(file,'rb') as file:
            translations = pickle.load(file)
        
        for k,s in tqdm(translations.items()):
            
            if k in translated:
                continue
            
            score = model.compute_score_split(s, None)
            score['tweet_id'] = k
            score['all'] =  model.compute_score(s, None)
            rr.append(score)
            
    if len(rr) >= 2500:
        df_scores = pd.DataFrame(rr)
        df_scores.to_pickle('_social_translations_' + str(int(datetime.now().timestamp())) + '.pickle')
        df_scores = pd.concat([df_scores.reset_index()['tweet_id'],pd.json_normalize(df_scores['all'])],axis=1).set_index('tweet_id')
        df_scores.to_pickle('_social_translations_simplified_' + str(int(datetime.now().timestamp())) + '.pickle')
        rr.clear()
#         break

if len(rr) > 0:
    df_scores = pd.DataFrame(rr)
    df_scores.to_pickle('_social_translations_' + str(int(datetime.now().timestamp())) + '.pickle')
    df_scores = pd.concat([df_scores.reset_index()['tweet_id'],pd.json_normalize(df_scores['all'])],axis=1).set_index('tweet_id')
    df_scores.to_pickle('_social_translations_simplified_' + str(int(datetime.now().timestamp())) + '.pickle')
    rr.clear()       

In [ ]:
for i in range(0,len(df_scores_reduced)):
    dd = df_scores_reduced.iloc[i].to_dict()
    print(' ## ',dd['text'])
    del dd['text']
    print(dict(sorted(dd.items(), key=lambda item: -item[1])))